In [ ]:
import os
import json

config_path = 'untappd.cfg'

with open(config_path) as rdr:
    config = json.load(rdr)

In [ ]:
import untappd_scraper
from untappd_scraper import ScraperType

browser = untappd_scraper.create_driver(config, headless=False)

In [18]:
## Identify users to scraper
import json
import time
import uuid
import feather

import pandas as pd
from glob import glob

## Identify all known users, and any scraped users from the list
reviews_df = pd.concat([feather.read_dataframe(file) for file in glob('../data/reviews*.feather')], sort=False)
scraped_user_ids = []
for file in glob('../data/users*.json'):
    print(file)
    with open(file) as rdr:
        scraped_user_ids.extend([x['username'] for x in json.load(rdr)])

users = set(reviews_df['user_id']) - set(scraped_user_ids)
len(users)

../data\users_a2d4a69d-a980-408e-b668-71a008fb3bee.json


214732

In [ ]:
import json
import uuid

users_data = []
review_results = []

user_scraper = untappd_scraper.UserScraper(browser)

for user in users:
    
    info, reviews = user_scraper.scrape_user(user)
    users_data.append(info)
    review_results.extend(reviews)
    
    print(user, len(reviews))
    if len(users_data) > 50:
        file_id = str(uuid.uuid4())

        # Write user reviews
        feather.write_dataframe(untappd_scraper.create_reviews_df(review_results), f'../data/reviews_{file_id}.feather')
        pd.DataFrame(review_results).to_json(f'../data/reviews_{file_id}.json', orient='records')

        # Write user info
        with open(f'../data/users_{file_id}.json', 'w') as wtr:
            json.dump(users_data, wtr)
            
        time.sleep(60)
    else:
        time.sleep(10)